# Data Merge

In [ ]:
import os
import pandas as pd
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
folder_path = "./Original data/"
save_path = "./"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
csv_files.sort() 

df_list = []
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, encoding='utf-8') 
    df_list.append(df)

df_merged = pd.concat(df_list, ignore_index=True)

output_csv = os.path.join(save_path, "./Relevant dataset/merged_data.csv")
df_merged.to_csv(output_csv, index=False, encoding='utf-8')



In [ ]:
file_path = "./Relevant dataset/merged_data.csv" 
df = pd.read_csv(file_path)

duplicate_rows = df[df.duplicated()]

print(duplicate_rows)

df_no_duplicates = df.drop_duplicates()

cleaned_file_path = "./Relevant dataset/merged_data_no_duplicates.csv" 
df_no_duplicates.to_csv(cleaned_file_path, index=False)



In [ ]:
file_path = "./Relevant dataset/merged_data.csv" 
df = pd.read_csv(file_path)

df['CASEVERSION'] = pd.to_numeric(df['CASEVERSION'], errors='coerce')

df_latest = df.loc[df.groupby('CASEID')['CASEVERSION'].idxmax()]

cleaned_file_path = "./Relevant dataset/merged_data_no_duplicates.csv"
df_latest.to_csv(cleaned_file_path, index=False)

In [ ]:
df = pd.read_csv("./Relevant dataset/merged_data_no_duplicates.csv")

hiv_keywords = ["HIV", "AIDS", "Human Immunodeficiency Virus", "Acquired immunodeficiency syndrome", "Immune reconstitution inflammatory syndrome associated tuberculosis", "Antiretroviral therapy"]

df_hiv_direct = df[df["INDI_PT"].str.contains('|'.join(hiv_keywords), case=False, na=False)]

df_hiv_direct.to_csv("./Relevant dataset/hiv_directly_related.csv", index=False)


In [ ]:
main_df = pd.read_csv("./Relevant dataset/hiv_directly_related.csv")
efv_mono_df = pd.read_csv("./Relevant dataset/only_EFV_drug_name.csv")       
efv_combo_df = pd.read_csv("./Relevant dataset/combin_drugnames.csv")       

main_df['DRUGNAME_UPPER'] = main_df['DRUGNAME'].str.upper()
efv_mono_set = set(efv_mono_df['DRUGNAME'].str.upper())
efv_combo_set = set(efv_combo_df['DRUGNAME'].str.upper())

efv_mono_records = main_df[main_df['DRUGNAME_UPPER'].isin(efv_mono_set)].drop(columns=['DRUGNAME_UPPER'])
efv_combo_records = main_df[main_df['DRUGNAME_UPPER'].isin(efv_combo_set)].drop(columns=['DRUGNAME_UPPER'])

efv_mono_records.to_csv("./Relevant dataset/efv_only_records.csv", index=False)
efv_combo_records.to_csv("./Relevant dataset/efv_combination_records.csv", index=False)


In [ ]:
file_path = "./Relevant dataset/efv_only_records.csv"
df = pd.read_csv(file_path)


if "PT" not in df.columns:
    raise ValueError("Cannot find PT")

descriptions = df["PT"].dropna().astype(str).tolist()

adverse_events = []
for desc in descriptions:
    events = re.split(r'[;,]', desc)  
    events = [event.strip().lower() for event in events if event.strip()] 
    adverse_events.extend(events)

adverse_event_counts = Counter(adverse_events)


df_event_freq = pd.DataFrame(adverse_event_counts.items(), columns=['Adverse Event', 'Count'])
df_event_freq = df_event_freq.sort_values(by="Count", ascending=False)

df_event_freq.to_csv("./Relevant dataset/adverse_event_frequencies.csv", index=False)

In [ ]:
df = pd.read_csv("./Relevant dataset/adverse_event_frequencies.csv")

df_filtered = df[df['Count'] >= 5]

df_filtered.to_csv("./Relevant dataset/adverse_events_count_ge_5.csv", index=False)

In [ ]:

ae_file      = './Relevant dataset/adverse_events_count_ge_5.csv'
map_file     = './Relevant dataset/pt_to_soc_map.csv'
output_file  = './Relevant dataset/ae_to_soc_cluster.csv'

ae = pd.read_csv(ae_file, dtype=str)
m  = pd.read_csv(map_file, dtype=str)


ae['AE_norm'] = ae['Adverse Event'].str.strip().str.lower()
m ['PT_norm'] = m ['PT_Term']        .str.strip().str.lower()

merged = ae.merge(
    m[['PT_norm','SOC_Term']],
    left_on='AE_norm',
    right_on='PT_norm',
    how='left'
)

result = merged[['Adverse Event','SOC_Term']].rename(
    columns={'SOC_Term':'SOC Cluster'}
)

result.to_csv(output_file, index=False, encoding='utf-8-sig')